In [7]:
from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator

In [ ]:
import numpy as np
import torch.nn as nn
import torch
from sklearn.metrics import accuracy_score

In [ ]:
@variational_estimator
class BayesianSAPLMA(nn.Module):
    def __init__(self, input_size):
        super(BayesianSAPLMA, self).__init__()

        self.network = nn.Sequential(
            BayesianLinear(input_size, 256),  
            nn.ReLU(),
            BayesianLinear(256, 128),        
            nn.ReLU(),
            BayesianLinear(128, 64),         
            nn.ReLU(),
            BayesianLinear(64, 1),           
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.network(x)

def train_classifier(classifier, train_loader, optimizer, criterion, epochs=5, device="cuda"):
    classifier.train()


    for epoch in range(epochs):
        for batch in train_loader:
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()           
            loss = classifier.sample_elbo(
                inputs=X_batch,
                labels=y_batch,
                criterion=lambda preds, targets: criterion(preds.squeeze(), targets),
                sample_nbr=10, 
            )
            loss.backward()
            optimizer.step()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


In [ ]:
def evaluate_classifier(classifier, test_loader, criterion, device="cuda", num_samples=10):
    classifier.eval() 
    total_loss = 0.0
    all_predictions = []
    all_labels = []

    with torch.no_grad():  
        for batch in test_loader:
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

           
            outputs = [classifier(X_batch) for _ in range(num_samples)]
            outputs = torch.stack(outputs)  
            pred_mean = outputs.mean(dim=0).flatten()  

           
            loss = criterion(pred_mean, y_batch)
            total_loss += loss.item()

            
            all_predictions.extend((pred_mean > 0.5).cpu().numpy()) 
            all_labels.extend(y_batch.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_predictions)

    return avg_loss, accuracy

In [ ]:
from sklearn.datasets import make_classification
import pandas as pd

# Generate synthetic data
X, y = make_classification(n_samples=20000,     # Number of samples
                           n_features=4096,     # Total number of features
                           n_informative=2000,  # Number of informative features
                           n_redundant=1000,     # Number of redundant features
                           n_classes=2,       # Number of classes (binary)
                           random_state=11)

# Convert to DataFrame for easier handling
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(4096)])
df['label'] = y

print(df.head())

ValueError: Number of informative, redundant and repeated features must sum to less than the number of total features

In [27]:
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split


X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)  

dataset = TensorDataset(X_tensor, y_tensor)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)  
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)   

In [28]:
torch.cuda.device_count()

1

In [29]:
# Model parameters
input_size = 4096  # Number of features

# Initialize model
model = BayesianSAPLMA(input_size=input_size)
model.to("cuda")  # Move to GPU if available

# Optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCELoss()  # Binary Cross-Entropy Loss

In [32]:
train_classifier(
    classifier=model,
    train_loader=train_loader,
    optimizer=optimizer,
    criterion=criterion,
    epochs=10,  # Number of epochs
    device="cuda"  # Use "cuda" if GPU is available, otherwise "cpu"
)


Epoch 1/10, Loss: 4436299.5
Epoch 2/10, Loss: 4384578.0
Epoch 3/10, Loss: 4332624.5
Epoch 4/10, Loss: 4281688.0
Epoch 5/10, Loss: 4229846.5
Epoch 6/10, Loss: 4178332.0
Epoch 7/10, Loss: 4126459.25
Epoch 8/10, Loss: 4074851.75
Epoch 9/10, Loss: 4023519.25
Epoch 10/10, Loss: 3972338.0


In [33]:
test_loss, test_accuracy = evaluate_classifier(
    classifier=model,
    test_loader=test_loader,
    criterion=criterion,
    device="cuda",
    num_samples=10
)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Loss: 0.6931
Test Accuracy: 50.05%
